In [21]:
import matplotlib.pyplot as plt


def latex_to_image(latex_str, filename="output.png"):
    """
    Renders a LaTeX expression to an image with a white background.

    Args:
    latex_str (str): The LaTeX string to render.
    filename (str): The filename where the image will be saved.
    """
    # Configure Matplotlib: use white background and remove axes
    plt.rcParams.update(
        {
            "text.usetex": True,
            "text.latex.preamble": r"\usepackage{amsmath}",
            "axes.facecolor": "white",
            "axes.edgecolor": "white",
            "axes.linewidth": 0,
            "xtick.bottom": False,
            "xtick.top": False,
            "ytick.left": False,
            "ytick.right": False,
            "xtick.labelbottom": False,
            "xtick.labeltop": False,
            "ytick.labelleft": False,
            "ytick.labelright": False,
        }
    )
    plt.figure(figsize=(2, 1))
    plt.text(
        0, 0, f"${latex_str}$", fontsize=40, ha="center", va="center", color="white"
    )
    plt.axis("off")
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1.5, bottom=0, right=0.5, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    # Plot color is white
    plt.gca().set_facecolor("white")

    # Save the image
    plt.savefig(filename, bbox_inches="tight", pad_inches=0, dpi=300)
    plt.close()


# Example usage
latex_statement = """\int_{-\infty}^{\infty} e^{-x^2} dx = \sqrt{\pi}"""
latex_to_image(latex_statement)

<>:49: SyntaxWarning: invalid escape sequence '\i'
<>:49: SyntaxWarning: invalid escape sequence '\i'
/var/folders/b_/lzmcr_wj73v_mycq99dq63nw0000gn/T/ipykernel_70665/425732541.py:49: SyntaxWarning: invalid escape sequence '\i'
  latex_statement = """\int_{-\infty}^{\infty} e^{-x^2} dx = \sqrt{\pi}"""


In [77]:
# Generate arbitrary LaTeX strings

# Singletons
numbers = [str(i) for i in range(100)]
# Alphabet
alphabet = [chr(i) for i in range(65, 91)] + [chr(i) for i in range(97, 123)]
# Greek alphabet LaTeX symbols (with \)
greek_lower = [
    "\\alpha",
    "\\beta",
    "\\gamma",
    "\\delta",
    "\\epsilon",
    "\\zeta",
    "\\eta",
    "\\theta",
    "\\iota",
    "\\kappa",
    "\\lambda",
    "\\mu",
    "\\nu",
    "\\xi",
    "\\omicron",
    "\\pi",
    "\\rho",
    "\\sigma",
    "\\tau",
    "\\upsilon",
    "\\phi",
    "\\chi",
    "\\psi",
    "\\omega",
]

greek_upper = [
    # "\\Alpha",
    # "\\Beta",
    # "\\Gamma",
    # "\\Delta",
    # "\\Epsilon",
    # "\\Zeta",
    # "\\Eta",
    # "\\Theta",
    # "\\Iota",
    # "\\Kappa",
    # "\\Lambda",
    # "\\Mu",
    # "\\Nu",
    # "\\Xi",
    # "\\Omicron",
    # "\\Pi",
    # "\\Rho",
    # "\\Sigma",
    # "\\Tau",
    # "\\Upsilon",
    # "\\Phi",
    # "\\Chi",
    # "\\Psi",
    # "\\Omega",
]

special_singleton = [
    "\\infty",
    "\\emptyset",
]

# Two element operators
two_op = [
    "#=#",
    "#\\neq#",
    "#\\approx#",
    "#\\equiv#",
    "#<#",
    "#\\leq#",
    "#>#",
    "#\\geq#",
    "#\\ll#",
    "#\\gg#",
    "#\\subset#",
    "#\\subseteq#",
    "#\\supset#",
    "#\\supseteq#",
    "#\\in#",
    "#\\notin#",
    "#\\ni#",
    "#\\notni#",
    "#\\cup#",
    "#\\cap#",
    "#\\setminus#",
    "#\\cdot#",
    "#\\times#",
    "#\\oplus#",
    "#\\otimes#",
    "#\\wedge#",
    "#\\vee#",
    "\\frac##",
    "#^#",
    "#_#",
    "(#)(#)",
    "(#)(#)",
    "(#)(#)",
    "(#)(#)",
    "(#)(#)",
    "(#)(#)",
    "(#)(#)",
    "(#)(#)",
    "(#)(#)",
    "(#)(#)",
]

# One element operators
one_op = [
    "\\forall#",
    "\\exists#",
    "\\nexists#",
    "\\neg#",
    "(#)",
    "[#]",
    "\\{#\\}",
    "\\lfloor#\\rfloor",
    "\\lceil#\\rceil",
    "\\langle#\\rangle",
    "\\partial#",
    "\\nabla#",
]

# Functions
functions = [
    "\\sin#",
    "\\cos#",
    "\\tan#",
    "\\cot#",
    "\\sec#",
    "\\csc#",
    "\\arcsin#",
    "\\arccos#",
    "\\arctan#",
    "\\sinh#",
    "\\cosh#",
    "\\tanh#",
    "\\coth#",
    "\\exp#",
    "\\log#",
    "\\ln#",
    "\\det#",
    "\\dim#",
    "\\ker#",
    "\\min#",
    "\\max#",
    "\\arg#",
    "\\lim#",
    "\\limsup#",
    "\\liminf#",
    "\\sup#",
    "\\inf#",
    "\\Pr#",
    "\\Re#",
    "\\Im#",
    "\\mod#",
    "\\floor#",
    "\\ceil#",
    "|#|",
    "\\argmax#",
    "\\argmin#",
    "\\sum#",
    "\\prod#",
    "\\int#",
    "\\oint#",
    "\\iint#",
    "\\iiint#",
    "\\iiiint#",
    "\\idotsint#",
    "\\bigcup#",
    "\\bigcap#",
    "\\bigvee#",
    "\\bigwedge#",
    "\\bigoplus#",
    "\\bigotimes#",
    "\\bigodot#",
    "\\biguplus#",
    "\\bigsqcup#",
    "\\biguplus#",
]

# Three element operators
three_op = [
    "\\int_#^##",
    "\\oint_#^##",
    "\\sum_#^##",
]

singletons = numbers + alphabet + greek_lower + greek_upper + special_singleton
one_ops = one_op + functions
two_ops = two_op
three_ops = three_op

# Preprocess: for each element, replace # with {#}
one_ops = [op.replace("#", "{#}") for op in one_ops]
two_ops = [op.replace("#", "{#}") for op in two_ops]
three_ops = [op.replace("#", "{#}") for op in three_ops]
# For each element in singletons, add space after it
singletons = [singleton + " " for singleton in singletons]

In [78]:
# Construct a random LaTeX expression
import random


# Methodology: Start with a #. Now until it has no more #, replace the first # with an expresson. Expression is singleton with probablity 0.5, one_op with probability 0.3, two_op with probability 0.1, three_op with probability 0.1
def generate_expression():
    expression = "#"
    while "#" in expression:
        rand = random.random()
        expressions = [
            random.choice(singletons),
            random.choice(one_ops),
            random.choice(two_ops),
            random.choice(three_ops),
        ]

        if rand < 0.4:
            expression = expression.replace("#", expressions[0], 1)
        elif rand < 0.8:
            expression = expression.replace("#", expressions[1], 1)
        elif rand < 0.95:
            expression = expression.replace("#", expressions[2], 1)
        else:
            expression = expression.replace("#", expressions[3], 1)
    return expression


expression = ""
while len(expression) < 400:
    expression = generate_expression()

print(expression)

\exists{\cosh{\sup{\oint_{\argmax{67 }}^{\argmin{\frac{\inf{\int_{\bigcap{\Im{67 }}}^{\max{\Re{c }}}{{\{{({\bigoplus{{{D }={\langle{77 }\rangle}}\cap{\emptyset }}})({\int_{{7 }\ni{55 }}^{\sinh{\sum{\lim{\int_{\limsup{\arctan{\tanh{\ln{\Re{\oint{g }}}}}}}^{\det{15 }}{72 }}}}}{[{\argmax{\sum_{E }^{78 }{Q }}}]}})}\}}\wedge{4 }}}}{H }}}{\sum_{45 }^{52 }{{\neg{n }}\leq{\bigcap{\csc{\arcsin{{\omega }<{47 }}}}}}}}}}
